# Day 6 - Lab 1: Building RAG Systems

**Objective:** Build a RAG (Retrieval-Augmented Generation) system orchestrated by LangGraph, scaling in complexity from a simple retriever to a multi-agent team that includes a grader and a router.

**Estimated Time:** 180 minutes

**Introduction:**
Welcome to Day 6! Today, we build one of the most powerful and common patterns for enterprise AI: a system that can answer questions about your private documents. We will use LangGraph to create a 'research team' of AI agents. Each agent will have a specific job, and LangGraph will act as the manager, orchestrating their collaboration to find the best possible answer.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We need several libraries for this lab. `langgraph` is the core orchestrator, `langchain` provides the building blocks, `faiss-cpu` is for our vector store, and `pypdf` is for loading documents.

**Model Selection:**
For RAG and agentic workflows, models with strong instruction-following and reasoning are best. `gpt-4.1`, `o3`, or `gemini-2.5-pro` are excellent choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `load_artifact()`: To read the project documents that will form our knowledge base.

In [1]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('langgraph')
install_if_missing('langchain')
install_if_missing('langchain_community')
install_if_missing('langchain_openai')
install_if_missing('faiss-cpu')
install_if_missing('pypdf')

from utils import setup_llm_client, load_artifact
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

faiss-cpu not found, installing...


2025-11-04 14:49:01,609 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: Building the Knowledge Base

An agent is only as smart as the information it can access. We will create a vector store containing all the project artifacts we've created so far. This will be our agent's 'knowledge base'.

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# Install sentence-transformers if not available
import subprocess
try:
    import sentence_transformers
except ImportError:
    print("Installing sentence-transformers...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "sentence-transformers"])

# Use HuggingFace embeddings - local and reliable
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize the local embeddings model
print("Loading embeddings model (this may take a moment on first run)...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("✅ Embeddings model loaded")

def create_knowledge_base(file_paths):
    """Loads documents from given paths and creates a FAISS vector store.""" 
    all_docs = []
    for path in file_paths:
        full_path = os.path.join(project_root, path)
        if os.path.exists(full_path):
            loader = TextLoader(full_path)
            docs = loader.load()
            for doc in docs:
                doc.metadata={"source": path} # Add source metadata
            all_docs.extend(docs)
        else:
            print(f"Warning: Artifact not found at {full_path}")

    if not all_docs:
        print("No documents found to create knowledge base.")
        return None

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(all_docs)
    
    print(f"Creating vector store from {len(splits)} document splits...")
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
    return vectorstore.as_retriever()

all_artifact_paths = ["docs/prd/day1_prd.md", "database/schema.sql", "docs/adr/adr_001_database_choice.md"]
retriever = create_knowledge_base(all_artifact_paths)
print("✅ Knowledge base created successfully!")

Loading embeddings model (this may take a moment on first run)...


/var/folders/18/2x6d6cb14xn9pwtdmrdcpsdh0000gp/T/ipykernel_85176/2985344843.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ Embeddings model loaded
Creating vector store from 43 document splits...
✅ Knowledge base created successfully!


## Step 3: The Challenges

### Challenge 1 (Foundational): A Simple RAG Graph

**Task:** Build a simple LangGraph with two nodes: one to retrieve documents and one to generate an answer.

> **Tip:** Think of `AgentState` as the shared 'whiteboard' for your agent team. Every agent (or 'node' in the graph) can read from and write to this state, allowing them to pass information to each other as they work on a problem.

**Instructions:**
1.  Define the state for your graph using a `TypedDict`. It should contain keys for `question` and `documents`.
2.  Create a "Retriever" node. This is a Python function that takes the state, uses the `retriever` to get relevant documents, and updates the state with the results.
3.  Create a "Generator" node. This function takes the state, creates a prompt with the question and retrieved documents, calls the LLM, and stores the answer.
4.  Build the `StateGraph`, add the nodes, and define the edges (`RETRIEVE` -> `GENERATE`).
5.  Compile the graph and invoke it with a question about your project.

**Expected Quality:** A functional graph that can answer a simple question (e.g., "What is the purpose of this project?") by retrieving context from the project artifacts.

In [3]:
# Challenge 1: Simple RAG Graph with LangGraph

# First, let's install and import required packages
import sys
import subprocess

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found, installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

# Install langchain-google-genai if using Google models
if api_provider == "google":
    install_if_missing('langchain-google-genai')

from typing import TypedDict
from langgraph.graph import StateGraph, END

print(f"🔧 Setting up RAG system with {api_provider} - {model_name}")

# Step 1: Verify the retriever exists and works
if retriever is None:
    print("❌ ERROR: Retriever is not initialized. Please run Step 2 first!")
    raise ValueError("Retriever not found")

print("✅ Retriever verified")

# Step 2: Define the state that will be passed between nodes
# Think of this as the "shared memory" for all nodes in the graph
class AgentState(TypedDict):
    question: str           # The user's question
    documents: list         # Retrieved documents will go here
    answer: str            # Final answer will go here

# Step 3: Initialize the LLM based on the provider
print(f"🤖 Initializing LLM...")
try:
    if api_provider == "google":
        from langchain_google_genai import ChatGoogleGenerativeAI
        llm = ChatGoogleGenerativeAI(model=model_name, temperature=0)
    elif api_provider == "openai":
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model=model_name, temperature=0)
    else:
        # For other providers, try ChatOpenAI as fallback
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model=model_name, temperature=0)
    print(f"✅ LLM initialized successfully")
except Exception as e:
    print(f"❌ Error initializing LLM: {e}")
    raise

# Step 4: Create the Retriever Node
def retrieve_node(state: AgentState) -> AgentState:
    """Retrieve relevant documents based on the question."""
    try:
        question = state["question"]
        print(f"\n🔍 RETRIEVE NODE: Searching for documents related to: '{question}'")
        
        # Use the retriever we created earlier to find relevant documents
        docs = retriever.invoke(question)
        
        print(f"   Found {len(docs)} relevant documents")
        
        # Return the updated state with the retrieved documents
        return {"documents": docs}
    except Exception as e:
        print(f"❌ Error in retrieve_node: {e}")
        return {"documents": []}

# Step 5: Create the Generator Node
def generate_node(state: AgentState) -> AgentState:
    """Generate an answer based on the question and retrieved documents."""
    try:
        question = state["question"]
        documents = state["documents"]
        
        if not documents:
            return {"answer": "I couldn't find any relevant documents to answer this question."}
        
        print(f"\n✍️  GENERATE NODE: Creating answer using {len(documents)} documents")
        
        # Format the documents into a context string
        context = "\n\n".join([doc.page_content for doc in documents])
        
        # Create a prompt that includes both the context and the question
        prompt = f"""Based on the following context, please answer the question.
        
Context:
{context}

Question: {question}

Answer:"""
        
        # Call the LLM to generate the answer
        response = llm.invoke(prompt)
        answer = response.content
        
        print(f"   Generated answer (preview): {answer[:100]}...")
        
        # Return the updated state with the answer
        return {"answer": answer}
    except Exception as e:
        print(f"❌ Error in generate_node: {e}")
        return {"answer": f"Error generating answer: {str(e)}"}

# Step 6: Build the StateGraph
print("\n🏗️  Building the RAG graph...")

# Create a new StateGraph with our AgentState
workflow = StateGraph(AgentState)

# Add our two nodes to the graph
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("generate", generate_node)

# Define the flow: Start -> Retrieve -> Generate -> End
workflow.set_entry_point("retrieve")  # Start with the retrieve node
workflow.add_edge("retrieve", "generate")  # After retrieve, go to generate
workflow.add_edge("generate", END)  # After generate, we're done

# Step 7: Compile the graph
app = workflow.compile()

print("✅ Graph compiled successfully!")

# Step 8: Test the graph with a question
print("\n" + "="*60)
print("TESTING THE RAG SYSTEM")
print("="*60)

test_question = "What is the purpose of this project?"

try:
    result = app.invoke({
        "question": test_question,
        "documents": [],
        "answer": ""
    })

    print("\n" + "="*60)
    print("FINAL RESULT")
    print("="*60)
    print(f"\nQuestion: {test_question}")
    print(f"\nAnswer: {result['answer']}")
    print(f"\nDocuments used: {len(result['documents'])}")
except Exception as e:
    print(f"\n❌ Error running the graph: {e}")
    import traceback
    traceback.print_exc()

langchain-google-genai not found, installing...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


🔧 Setting up RAG system with google - gemini-2.5-pro
✅ Retriever verified
🤖 Initializing LLM...
✅ LLM initialized successfully

🏗️  Building the RAG graph...
✅ Graph compiled successfully!

TESTING THE RAG SYSTEM

🔍 RETRIEVE NODE: Searching for documents related to: 'What is the purpose of this project?'
   Found 4 relevant documents

✍️  GENERATE NODE: Creating answer using 4 documents
   Generated answer (preview): Based on the context provided, the purpose of this project is to improve the company's new hire onbo...

FINAL RESULT

Question: What is the purpose of this project?

Answer: Based on the context provided, the purpose of this project is to improve the company's new hire onboarding process by achieving four main goals:

1.  **Accelerate New Hire Time-to-Productivity:** To get new employees contributing effectively in a shorter amount of time.
2.  **Enhance the New Hire Experience:** To improve satisfaction for new employees during their first 30 days.
3.  **Increase Operati

### Challenge 2 (Intermediate): A Graph with a Grader Agent

**Task:** Add a second agent to your graph that acts as a "Grader," deciding if the retrieved documents are relevant enough to answer the question.

> **What is a conditional edge?** It's a decision point. After a node completes its task (like our 'Grader'), the conditional edge runs a function to decide which node to go to next. This allows your agent to change its plan based on new information.

**Instructions:**
1.  Keep your `RETRIEVE` and `GENERATE` nodes from the previous challenge.
2.  Create a new "Grader" node. This function takes the state (question and documents) and calls an LLM with a specific prompt: "Based on the question and the following documents, is the information sufficient to answer the question? Answer with only 'yes' or 'no'."
3.  Add a **conditional edge** to your graph. After the `RETRIEVE` node, the graph should go to the `GRADE` node. After the `GRADE` node, it should check the grader's response. If 'yes', it proceeds to the `GENERATE` node. If 'no', it goes to an `END` node, concluding that it cannot answer the question.

**Expected Quality:** A more robust graph that can gracefully handle cases where its knowledge base doesn't contain the answer, preventing it from hallucinating.

In [4]:
# Challenge 2: RAG Graph with Grader Agent and Conditional Edges

from typing import TypedDict
from langgraph.graph import StateGraph, END

print("🏗️  Building RAG system with Grader agent...")

# Step 1: Define the AgentState (same as Challenge 1, but adding grade field)
class AgentState(TypedDict):
    question: str           # The user's question
    documents: list         # Retrieved documents
    grade: str             # Grader's decision: 'yes' or 'no'
    answer: str            # Final answer

# Step 2: Initialize the LLM (reusing from Challenge 1)
if api_provider == "google":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model=model_name, temperature=0)
elif api_provider == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model=model_name, temperature=0)
else:
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model=model_name, temperature=0)

print("✅ LLM initialized")

# Step 3: Create the Retriever Node (same as Challenge 1)
def retrieve_node(state: AgentState) -> AgentState:
    """Retrieve relevant documents based on the question."""
    question = state["question"]
    print(f"\n🔍 RETRIEVE NODE: Searching for: '{question}'")
    
    docs = retriever.invoke(question)
    print(f"   Found {len(docs)} documents")
    
    return {"documents": docs}

# Step 4: Create the NEW Grader Node
# This agent evaluates if the retrieved documents are good enough to answer the question
def grade_node(state: AgentState) -> AgentState:
    """Grade whether the retrieved documents are relevant to the question."""
    question = state["question"]
    documents = state["documents"]
    
    print(f"\n📊 GRADE NODE: Evaluating document relevance...")
    
    # Create a context string from the documents
    context = "\n\n".join([doc.page_content[:200] for doc in documents])  # Truncate for grading
    
    # Ask the LLM to grade the documents
    grade_prompt = f"""Based on the question and the following documents, is the information sufficient to answer the question?

Question: {question}

Documents:
{context}

Answer with ONLY 'yes' or 'no' (lowercase, no punctuation)."""
    
    response = llm.invoke(grade_prompt)
    grade = response.content.strip().lower()
    
    # Ensure we got a valid response
    if grade not in ['yes', 'no']:
        # Default to 'no' if unclear response
        print(f"   Unclear grade response: '{grade}', defaulting to 'no'")
        grade = 'no'
    
    print(f"   Grade: {grade.upper()}")
    
    return {"grade": grade}

# Step 5: Create the Generator Node (same as Challenge 1)
def generate_node(state: AgentState) -> AgentState:
    """Generate an answer based on the question and retrieved documents."""
    question = state["question"]
    documents = state["documents"]
    
    print(f"\n✍️  GENERATE NODE: Creating answer...")
    
    context = "\n\n".join([doc.page_content for doc in documents])
    
    prompt = f"""Based on the following context, please answer the question.
    
Context:
{context}

Question: {question}

Answer:"""
    
    response = llm.invoke(prompt)
    answer = response.content
    
    print(f"   Answer generated (preview): {answer[:80]}...")
    
    return {"answer": answer}

# Step 6: Create a function to decide where to route after grading
# This is the "conditional edge" logic
def decide_to_generate(state: AgentState) -> str:
    """
    Conditional edge function that determines the next node based on the grade.
    
    Returns:
        - "generate" if grade is 'yes' (documents are good)
        - "end" if grade is 'no' (documents are insufficient)
    """
    grade = state.get("grade", "no")
    
    if grade == "yes":
        print("   → Routing to GENERATE (documents are sufficient)")
        return "generate"
    else:
        print("   → Routing to END (documents are insufficient)")
        return "end"

# Step 7: Build the StateGraph with conditional edges
workflow = StateGraph(AgentState)

# Add all three nodes
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("grade", grade_node)
workflow.add_node("generate", generate_node)

# Define the flow with conditional routing:
# Start → Retrieve → Grade → (conditional: Generate or End)
workflow.set_entry_point("retrieve")

# After retrieve, always go to grade
workflow.add_edge("retrieve", "grade")

# After grade, use conditional edge to decide next step
# The decide_to_generate function returns either "generate" or "end"
workflow.add_conditional_edges(
    "grade",                    # From the grade node
    decide_to_generate,         # Use this function to decide where to go
    {
        "generate": "generate",  # If function returns "generate", go to generate node
        "end": END              # If function returns "end", end the workflow
    }
)

# After generate, we're done
workflow.add_edge("generate", END)

# Step 8: Compile the graph
app = workflow.compile()

print("✅ Graph with Grader compiled successfully!")

# Step 9: Test with a question that SHOULD have good documents
print("\n" + "="*70)
print("TEST 1: Question with relevant documents")
print("="*70)

test_question_1 = "What database was chosen for this project?"

result_1 = app.invoke({
    "question": test_question_1,
    "documents": [],
    "grade": "",
    "answer": ""
})

print("\n" + "="*70)
print("RESULT 1")
print("="*70)
print(f"Question: {test_question_1}")
print(f"Grade: {result_1.get('grade', 'N/A')}")
print(f"Answer: {result_1.get('answer', 'No answer generated - insufficient documents')}")

# Step 10: Test with a question that should NOT have good documents
print("\n" + "="*70)
print("TEST 2: Question with irrelevant documents")
print("="*70)

test_question_2 = "What is the weather like on Mars?"

result_2 = app.invoke({
    "question": test_question_2,
    "documents": [],
    "grade": "",
    "answer": ""
})

print("\n" + "="*70)
print("RESULT 2")
print("="*70)
print(f"Question: {test_question_2}")
print(f"Grade: {result_2.get('grade', 'N/A')}")
print(f"Answer: {result_2.get('answer', 'No answer generated - insufficient documents')}")

print("\n✅ Challenge 2 complete! The Grader prevents hallucinations by checking document relevance.")

🏗️  Building RAG system with Grader agent...
✅ LLM initialized
✅ Graph with Grader compiled successfully!

TEST 1: Question with relevant documents

🔍 RETRIEVE NODE: Searching for: 'What database was chosen for this project?'
   Found 4 documents

📊 GRADE NODE: Evaluating document relevance...
   Grade: YES
   → Routing to GENERATE (documents are sufficient)

✍️  GENERATE NODE: Creating answer...
   Answer generated (preview): Based on the context provided, the database chosen for the project is **PostgreS...

RESULT 1
Question: What database was chosen for this project?
Grade: yes
Answer: Based on the context provided, the database chosen for the project is **PostgreSQL with the `pgvector` extension**.

TEST 2: Question with irrelevant documents

🔍 RETRIEVE NODE: Searching for: 'What is the weather like on Mars?'
   Found 4 documents

📊 GRADE NODE: Evaluating document relevance...
   Grade: NO
   → Routing to END (documents are insufficient)

RESULT 2
Question: What is the weather lik

### Challenge 3 (Advanced): A Multi-Agent Research Team

**Task:** Build a sophisticated "research team" of specialized agents that includes a router to delegate tasks to the correct specialist.

**Instructions:**
1.  **Specialize your retriever:** Create two separate retrievers. One for the PRD (`prd_retriever`) and one for the technical documents (`tech_retriever` for schema and ADRs).
2.  **Define the Agents:**
    * `ProjectManagerAgent`: This will be the entry point and will act as a router. It uses an LLM to decide whether the user's question is about product requirements or technical details, and routes to the appropriate researcher.
    * `PRDResearcherAgent`: A node that uses the `prd_retriever`.
    * `TechResearcherAgent`: A node that uses the `tech_retriever`.
    * `SynthesizerAgent`: A node that takes the collected documents from either researcher and synthesizes a final answer.
3.  **Build the Graph:** Use conditional edges to orchestrate the flow: The entry point is the `ProjectManager`, which then routes to either the `PRD_RESEARCHER` or `TECH_RESEARCHER`. Both of those nodes should then route to the `SYNTHESIZE` node, which then goes to the `END`.

**Expected Quality:** A highly advanced agentic system that mimics a real-world research workflow, including a router and specialist roles, to improve the accuracy and efficiency of the RAG process.

In [5]:
# Challenge 3: Multi-Agent Research Team with Specialized Retrievers and Router

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END

print("🏗️  Building Multi-Agent Research Team...")

# Step 1: Create specialized retrievers for different document types
print("\n📚 Creating specialized retrievers...")

# PRD Retriever - only searches product requirement documents
def create_prd_retriever():
    """Creates a retriever specialized for PRD documents."""
    from langchain_community.vectorstores import FAISS
    from langchain_community.document_loaders import TextLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    
    prd_paths = ["docs/prd/day1_prd.md"]
    docs = []
    
    for path in prd_paths:
        full_path = os.path.join(project_root, path)
        if os.path.exists(full_path):
            loader = TextLoader(full_path)
            loaded_docs = loader.load()
            for doc in loaded_docs:
                doc.metadata = {"source": path, "type": "prd"}
            docs.extend(loaded_docs)
    
    if docs:
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = splitter.split_documents(docs)
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        print(f"   ✅ PRD Retriever created with {len(splits)} chunks")
        return vectorstore.as_retriever()
    return None

# Tech Retriever - only searches technical documents (schema, ADRs)
def create_tech_retriever():
    """Creates a retriever specialized for technical documents."""
    from langchain_community.vectorstores import FAISS
    from langchain_community.document_loaders import TextLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    
    tech_paths = ["database/schema.sql", "docs/adr/adr_001_database_choice.md"]
    docs = []
    
    for path in tech_paths:
        full_path = os.path.join(project_root, path)
        if os.path.exists(full_path):
            loader = TextLoader(full_path)
            loaded_docs = loader.load()
            for doc in loaded_docs:
                doc.metadata = {"source": path, "type": "technical"}
            docs.extend(loaded_docs)
    
    if docs:
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = splitter.split_documents(docs)
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        print(f"   ✅ Tech Retriever created with {len(splits)} chunks")
        return vectorstore.as_retriever()
    return None

# Create both specialized retrievers
prd_retriever = create_prd_retriever()
tech_retriever = create_tech_retriever()

# Step 2: Define the AgentState for the multi-agent system
class ResearchState(TypedDict):
    question: str               # The user's question
    route: str                  # Router's decision: 'prd' or 'technical'
    documents: list             # Documents retrieved by specialist
    answer: str                 # Final synthesized answer

# Step 3: Initialize the LLM
if api_provider == "google":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model=model_name, temperature=0)
elif api_provider == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model=model_name, temperature=0)
else:
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model=model_name, temperature=0)

print("✅ LLM initialized for multi-agent system")

# Step 4: Create the Project Manager Agent (Router)
def project_manager_node(state: ResearchState) -> ResearchState:
    """
    Acts as a router to determine if the question is about:
    - Product requirements (PRD) 
    - Technical implementation (database, architecture)
    """
    question = state["question"]
    
    print(f"\n👔 PROJECT MANAGER: Analyzing question type...")
    
    # Ask LLM to classify the question
    routing_prompt = f"""You are a project manager routing questions to the right specialist.

Question: {question}

Is this question about:
A) Product requirements, features, business goals, or user needs (answer: 'prd')
B) Technical implementation, database, architecture, or technical decisions (answer: 'technical')

Answer with ONLY 'prd' or 'technical' (lowercase, no punctuation)."""
    
    response = llm.invoke(routing_prompt)
    route = response.content.strip().lower()
    
    # Validate response
    if route not in ['prd', 'technical']:
        print(f"   Unclear route: '{route}', defaulting to 'technical'")
        route = 'technical'
    
    print(f"   📋 Routing to: {route.upper()} specialist")
    
    return {"route": route}

# Step 5: Create the PRD Researcher Agent
def prd_researcher_node(state: ResearchState) -> ResearchState:
    """Specialist agent for product requirement questions."""
    question = state["question"]
    
    print(f"\n📊 PRD RESEARCHER: Searching product documents...")
    
    if prd_retriever:
        docs = prd_retriever.invoke(question)
        print(f"   Found {len(docs)} PRD documents")
        return {"documents": docs}
    else:
        print("   ⚠️  No PRD retriever available")
        return {"documents": []}

# Step 6: Create the Tech Researcher Agent
def tech_researcher_node(state: ResearchState) -> ResearchState:
    """Specialist agent for technical questions."""
    question = state["question"]
    
    print(f"\n🔧 TECH RESEARCHER: Searching technical documents...")
    
    if tech_retriever:
        docs = tech_retriever.invoke(question)
        print(f"   Found {len(docs)} technical documents")
        return {"documents": docs}
    else:
        print("   ⚠️  No tech retriever available")
        return {"documents": []}

# Step 7: Create the Synthesizer Agent
def synthesizer_node(state: ResearchState) -> ResearchState:
    """
    Takes documents from either researcher and synthesizes a final answer.
    This is the final step where we generate the answer.
    """
    question = state["question"]
    documents = state["documents"]
    route = state.get("route", "unknown")
    
    print(f"\n🎯 SYNTHESIZER: Creating final answer from {route} specialist...")
    
    if not documents:
        return {"answer": f"I couldn't find sufficient {route} information to answer this question."}
    
    # Create context from documents
    context = "\n\n".join([doc.page_content for doc in documents])
    
    # Generate comprehensive answer
    synthesis_prompt = f"""You are a synthesizer agent. Based on the {route} documents provided, give a comprehensive answer.

Question: {question}

Context from {route} specialist:
{context}

Provide a clear, detailed answer:"""
    
    response = llm.invoke(synthesis_prompt)
    answer = response.content
    
    print(f"   ✅ Answer synthesized (preview): {answer[:80]}...")
    
    return {"answer": answer}

# Step 8: Create routing function for conditional edges
def route_question(state: ResearchState) -> Literal["prd_researcher", "tech_researcher"]:
    """
    Determines which researcher to route to based on the project manager's decision.
    
    Returns:
        - "prd_researcher" if question is about product requirements
        - "tech_researcher" if question is about technical details
    """
    route = state.get("route", "technical")
    
    if route == "prd":
        return "prd_researcher"
    else:
        return "tech_researcher"

# Step 9: Build the Multi-Agent StateGraph
print("\n🔨 Assembling the research team...")

workflow = StateGraph(ResearchState)

# Add all agent nodes
workflow.add_node("project_manager", project_manager_node)
workflow.add_node("prd_researcher", prd_researcher_node)
workflow.add_node("tech_researcher", tech_researcher_node)
workflow.add_node("synthesizer", synthesizer_node)

# Define the workflow:
# Start → Project Manager (router) → [PRD or Tech Researcher] → Synthesizer → End
workflow.set_entry_point("project_manager")

# After project manager, route conditionally to the appropriate researcher
workflow.add_conditional_edges(
    "project_manager",
    route_question,
    {
        "prd_researcher": "prd_researcher",
        "tech_researcher": "tech_researcher"
    }
)

# Both researchers route to synthesizer
workflow.add_edge("prd_researcher", "synthesizer")
workflow.add_edge("tech_researcher", "synthesizer")

# Synthesizer ends the workflow
workflow.add_edge("synthesizer", END)

# Step 10: Compile the graph
app = workflow.compile()

print("✅ Multi-Agent Research Team assembled!")

# Step 11: Test with a PRD question
print("\n" + "="*70)
print("TEST 1: Product Requirements Question")
print("="*70)

prd_question = "What are the main goals of this project?"

result_1 = app.invoke({
    "question": prd_question,
    "route": "",
    "documents": [],
    "answer": ""
})

print("\n" + "="*70)
print("RESULT 1")
print("="*70)
print(f"Question: {prd_question}")
print(f"Routed to: {result_1.get('route', 'N/A').upper()}")
print(f"Documents found: {len(result_1.get('documents', []))}")
print(f"\nAnswer:\n{result_1.get('answer', 'No answer generated')}")

# Step 12: Test with a technical question
print("\n" + "="*70)
print("TEST 2: Technical Question")
print("="*70)

tech_question = "What database technology was selected and why?"

result_2 = app.invoke({
    "question": tech_question,
    "route": "",
    "documents": [],
    "answer": ""
})

print("\n" + "="*70)
print("RESULT 2")
print("="*70)
print(f"Question: {tech_question}")
print(f"Routed to: {result_2.get('route', 'N/A').upper()}")
print(f"Documents found: {len(result_2.get('documents', []))}")
print(f"\nAnswer:\n{result_2.get('answer', 'No answer generated')}")

print("\n" + "="*70)
print("✅ Challenge 3 Complete!")
print("="*70)
print("You've built a sophisticated multi-agent research team that:")
print("  1. Routes questions to specialized researchers")
print("  2. Uses domain-specific retrievers")
print("  3. Synthesizes comprehensive answers")
print("  4. Mimics real-world research workflows")
print("="*70)

🏗️  Building Multi-Agent Research Team...

📚 Creating specialized retrievers...
   ✅ PRD Retriever created with 29 chunks
   ✅ Tech Retriever created with 14 chunks
✅ LLM initialized for multi-agent system

🔨 Assembling the research team...
✅ Multi-Agent Research Team assembled!

TEST 1: Product Requirements Question

👔 PROJECT MANAGER: Analyzing question type...
   📋 Routing to: PRD specialist

📊 PRD RESEARCHER: Searching product documents...
   Found 4 PRD documents

🎯 SYNTHESIZER: Creating final answer from prd specialist...
   ✅ Answer synthesized (preview): Based on the provided PRD documents, the main goals of this project are to creat...

RESULT 1
Question: What are the main goals of this project?
Routed to: PRD
Documents found: 4

Answer:
Based on the provided PRD documents, the main goals of this project are to create a centralized and measurable onboarding system that improves outcomes for new hires, increases efficiency for the company, and ensures regulatory compliance.

He

## Lab Conclusion

Incredible work! You have now built a truly sophisticated AI system. You've learned how to create a knowledge base for an agent and how to use LangGraph to orchestrate a team of specialized agents to solve a complex problem. You progressed from a simple RAG chain to a system that includes quality checks (the Grader) and intelligent task delegation (the Router). These are the core patterns for building production-ready RAG applications.

> **Key Takeaway:** LangGraph allows you to define complex, stateful, multi-agent workflows as a graph. Using nodes for agents and conditional edges for decision-making enables the creation of sophisticated systems that can reason, delegate, and collaborate to solve problems more effectively than a single agent could alone.